In [11]:
import pandas as pd
import numpy as np

In [12]:
def calc_marginal_concordances(el1, el2, info):
    cs = []
    for i, (a, b) in enumerate(zip(el1, el2)):
        t, q, p, _ = info[i].values()
        if t == 'g':
            if b < a + q:
                cs.append(1)
            elif b < a + p:
                cs.append((a+p-b)/(p-q))
            else:
                cs.append(0)
        else: # criterion type cost
            if b < a - p:
                cs.append(0)
            elif b < a - q:
                cs.append((b-a+p)/(p-q))
            else:
                cs.append(1)
    return cs

def calc_marginal_discordances(el1, el2, info):
    ds = []
    for i, (a, b) in enumerate(zip(el1, el2)):
        t, _, p, v = info[i].values()
        if t == 'g':
            if b < a + p:
                ds.append(0)
            elif b < a + v:
                ds.append((b-a-p)/(v-p))
            else:
                ds.append(1)
        else: # criterion type cost
            if b < a - v:
                ds.append(1)
            elif b < a - p:
                ds.append((a-p-b)/(v-p))
            else:
                ds.append(0)
    return ds

def comprehensive_concordance(marginal_conc, weights):
    return np.dot(np.array(weights), np.array(marginal_conc)) / np.sum(weights)

def outranking_credibilities_sigma(conc_C: float, dis_D):
    return conc_C * np.prod([(1-dis)/(1-conc_C) for dis in dis_D if dis > conc_C])

def create_credibility_mat(df, weights, info):
    arr = np.array(df)
    credibility_mat = np.zeros((arr.shape[0],arr.shape[0]))
    for i in range(arr.shape[0]):
        for j in range(arr.shape[0]):
            c = calc_marginal_concordances(arr[i], arr[j], info)
            D = calc_marginal_discordances(arr[i], arr[j], info)
            C = comprehensive_concordance(c, weights)
            credibility_mat[i,j] = outranking_credibilities_sigma(C, D)

    return credibility_mat

In [13]:
df = pd.read_csv("przyklad_wyk.csv")
weights = [2, 3]
info = [
    {'type': 'g', 'iff': 10, 'pref':50, 'veto':100},
    {'type': 'c', 'iff': 0, 'pref':10, 'veto':20} 
]
credibility_mat = create_credibility_mat(df, weights, info)
credibility_mat


array([[1. , 0. , 0. ],
       [1. , 1. , 0.7],
       [1. , 0.3, 1. ]])

In [14]:
df = pd.read_csv("przyklad_lab.csv")
weights = [3, 3, 4]
info = [
    {'type': 'g', 'iff': 4, 'pref':12, 'veto':28},
    {'type': 'g', 'iff': 1, 'pref':2, 'veto':8},
    {'type': 'c', 'iff': 100, 'pref':200, 'veto':600} 
]
credibility_mat = create_credibility_mat(df, weights, info)
credibility_mat

array([[1.        , 0.6       , 0.3       , 0.7       , 0.55      ],
       [0.        , 1.        , 0.2037037 , 0.        , 0.        ],
       [0.58333333, 0.6       , 1.        , 0.85      , 0.        ],
       [0.3       , 0.        , 0.6       , 1.        , 0.58333333],
       [0.6       , 0.        , 0.6       , 0.7       , 1.        ]])

In [15]:
def get_rank(element, rank):
    for i, item in enumerate(rank):
        if isinstance(item, list):
            if element in item:
                return i
        elif element == item:
            return i

def is_above(a, b, rank1, rank2):
    a_rank1 = get_rank(a,rank1)
    b_rank1 = get_rank(b,rank1)

    a_rank2 = get_rank(a,rank2)
    b_rank2 = get_rank(b,rank2)

    if a_rank1 >= b_rank1 and a_rank2 >= b_rank2:
        return True
    

def make_outranking_list(asc_rank, des_rank, elements: list):
    result = {}

    # Iteracja po elementach w rankings
    for element in elements:
        result[element] = []
        for item in elements:
            if is_above(element,item, asc_rank, des_rank):
                result[element].append(item)

    # Wyświetlanie wyników
    # for element, elements_above in result.items():
    #     print(f"Element {element} jest nad: {elements_above}")

    print("pair = {", end="")
    for element, elements_above in result.items():
        for above in elements_above:
                # print("{"+f"{elements.index(element)+1},{elements.index(above)+1}"+"}, ",end="")
                print("{"+f"\"{element}\",\"{above}\""+"}, ",end="")

    print("}")

def syg(value, alfa = -0.15, beta = 0.3):
    return alfa*value + beta

def destilate(S, lamb = None, names = None, is_descending = True, alfa = -0.15, beta = 0.3):
    if S.shape[0] == 0:
        return []

    S_ = np.copy(S)
    np.fill_diagonal(S_, 0)

    if lamb is None:
        lamb = np.max(S_)
    
    if lamb == 0:
        if len(names) == 1:
            return list(names)
        return [list(names)]
    
    if names is None:
        names = np.array([i for i in range(S.shape[0])])
    S_[S_ >= lamb - syg(lamb,alfa,beta)] = 0
    lamb_min = max(np.max(S_), 0)
    
    survivors = np.copy(S)
    np.fill_diagonal(survivors, 0)
    survivors[survivors <= lamb_min] = 0


    for a in range(survivors.shape[0]):
        for b in range(survivors.shape[1]):
            # print(S[a][b], S[b][a], syg(S[a][b]))
            if S[a][b] <= S[b][a] + syg(S[a][b]):
                survivors[a][b] = 0
    s = np.sum(survivors>0,axis=1)
    w = np.sum(survivors>0,axis=0)
    q = s - w
    if is_descending:
        win_score = np.max(q)
    else:
        win_score = np.min(q)
    mask = q == win_score
    if np.sum(mask) > 1: # internal destilation
        choice = destilate(S[mask][:,mask],lamb_min,names[mask],is_descending, alfa, beta)
    else:
        choice = [names[mask][0]]
    ret = destilate(S[~mask][:,~mask], None, names[~mask],is_descending, alfa, beta)

    return  ret + choice

def descending_distillation(S,names):
    return destilate(S,None, np.array(names), is_descending= True)

def ascending_distillation(S,names):
    return destilate(S,None, np.array(names), is_descending= False)[::-1]

In [17]:

asc_rank = ascending_distillation(credibility_mat,["I", "B", "G", "A", "F"])
des_rank = descending_distillation(credibility_mat,["I", "B", "G", "A", "F"])

print(asc_rank)
print(des_rank)

['A', 'B', 'I', 'G', 'F']
[['B', 'A', 'F'], 'I', 'G']


In [34]:
df = pd.read_csv("ranking_telefony.csv")
names = df.loc[:,'nazwa']
df = df.drop(['nazwa'], axis=1)

weights = [3, 1, 2, 2, 3]
info = [
    {'type': 'c', 'iff': 300, 'pref':800, 'veto':1500},
    {'type': 'g', 'iff': 0.2, 'pref':0.5, 'veto':1},
    {'type': 'g', 'iff': 250, 'pref':500, 'veto':1000},
    {'type': 'g', 'iff': 32, 'pref':128, 'veto':150},
    {'type': 'g', 'iff': 2, 'pref':4, 'veto':6} 
]

credibility_mat = create_credibility_mat(df, weights, info)
# print(credibility_mat)

asc_rank = ascending_distillation(credibility_mat,names)
des_rank = descending_distillation(credibility_mat,names)
# ranks = make_outranking_list(asc_rank,des_rank, names)

print(asc_rank)
print(des_rank)
# print(list(names))

['Huawei Nova 9', ['Realme GT Master Edition ', 'Xiaomi 11 Lite Ne'], 'Samsung Galaxy S21', 'Vivo V21', 'Sony Xperia 10 III', 'Xiaomi Redmi 10', ['Oppo A16', 'Motorola G50'], 'Xiaomi POCO F3', 'Sony Xperia 1 III', ['IPhone 11 Pro', 'DOOGEE S59', 'Xiaomi Poco X3 Pro']]
[['Sony Xperia 1 III', 'IPhone 11 Pro'], 'Samsung Galaxy S21', 'DOOGEE S59', 'Oppo A16', 'Huawei Nova 9', ['Vivo V21', 'Realme GT Master Edition ', 'Xiaomi 11 Lite Ne'], 'Sony Xperia 10 III', ['Motorola G50', 'Xiaomi Redmi 10'], 'Xiaomi POCO F3', 'Xiaomi Poco X3 Pro']
